# Домашнее задание № 3. Исправление опечаток

In [1]:
!pip install textdistance

In [2]:
import textdistance
from pprint import pprint
from string import punctuation
punctuation += "«»—…“”"
punct = set(punctuation)
from collections import Counter
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [4]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()
corpus = open('wiki_data.txt', encoding='utf8').read()

vocab = Counter(re.findall('\w+', corpus.lower()))

In [5]:
word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=1000)
X = vec.fit_transform(vocab)

In [6]:
X.shape

(368802, 1000)

In [7]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость 
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

In [8]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

In [9]:
def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    
    return closest

In [10]:
N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

In [11]:
get_closest_hybrid_match("сомнце", X, vec)

[('солнце', 0.8333333333333334),
 ('солнцем', 0.7142857142857143),
 ('солнцев', 0.7142857142857143)]

In [12]:
def one_right(word, iks, vect):
    probabilities = {p[0]:P(p[0]) for p in get_closest_hybrid_match(word, iks, vect)}
    highest = max(probabilities.values())
    right_choice = str(list(probabilities.keys())[list(probabilities.values()).index(highest)])
    print(right_choice)    

In [13]:
one_right('солнце', X, vec)

солнце


In [14]:
one_right('сомнце', X, vec)

солнце


In [15]:
one_right('приветт', X, vec)

привет


In [16]:
one_right('симпотичный', X, vec)

симпатичных


## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [17]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [18]:
corpus = open('wiki_data.txt', encoding='utf8').read()
vocab0 = Counter(re.findall('\w+', corpus.lower()))
WORDS = list(vocab0)

In [19]:
N = sum(vocab0.values())

def P(word, N=N):
    return vocab0[word] / N

In [20]:
def correction(w):
    if w in WORDS:
        return w
    elif w in dict_del:
        return max(dict_del[w], key=P)
    else:
        opts = list()
        edits1(w)
        for ud in dels:
            if ud in dict_del:
                opts += dict_del[ud]
        if len(opts) > 0:
            return max(opts, key=P)
        else:
            return w

def edits1(word):
    global dels
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    dels = set(deletes)
    return dels

def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

dict_del = {}
for WORD in WORDS:
    edits1(WORD)
    for dell in dels:
        if dell in dict_del:
            dict_del[dell].append(WORD)
        else:
            dict_del[dell] = [WORD]

In [21]:
correction('солмце')

'солнце'

In [22]:
correction('мсг')

'мог'

In [23]:
%time
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        
        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[1]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)

Wall time: 0 ns
0
100
200
300
400
500
600
700
800
900


In [24]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8726363181590795
0.3423913043478261
0.04892615137245894
